In [7]:
import multiprocessing as mp
import numpy as np
import torch

from scipy.spatial.transform import Rotation
from rrc.env import fop, make_env
from trifinger_simulation.tasks.move_cube import Pose

In [2]:
env = make_env.env_fn_generator(diff=1, env_cls='robot_wrench_env', flatten_goal=True)()
obs = env.reset()
obs, r, d ,i = env.step(np.zeros(6))

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/scr-ssd/ksrini/trifinger_simulation/python/trifinger_simulation/trifinger_platform.py:293: UserWarning: Cameras are not enabled, so images in the camera observation are not initialized.  Create `TriFingerPlatform` with `enable_cameras=True` to get rendered camera images.
  "Cameras are not enabled, so images in the camera observation"


In [3]:
cp_wf = env.get_cp_wf_list(None, obj_pose=Pose(obs['achieved_goal'][:3], obs['achieved_goal'][3:]), use_actual_cp=True)
cp_of = env.get_cp_of_list(None, obj_pose=Pose(obs['achieved_goal'][:3], obs['achieved_goal'][3:]), use_actual_cp=True)

cp_v = []
for cpwf in cp_wf:
    if cpwf is None:
        cp_v.append(np.zeros(7))
    else:
        cp_v.append(np.concatenate(cpwf))
cp_v = np.array(cp_v)

In [4]:
%time x=np.apply_along_axis(env.fop.get_grasp_matrix, 1, cp_list)

NameError: name 'cp_list' is not defined

In [12]:
acscale = np.concatenate([env.force_factor, [env.torque_factor]*3])

In [13]:
%%time 
n = 2048
cp_list = np.array([cp_v.flatten() for i in range(n)])
des_wrench_t = torch.tensor([env.action_space.sample().astype('float32') * acscale for _ in range(n)]).to('cuda:0')
lamb = env.fop(des_wrench_t, cp_list)

ValueError: Two or more parameters have different dtypes. Expected parameter 1 to have dtype torch.float32 but got dtype torch.float64.

In [6]:
lamb.

torch.Size([2048, 9])

In [5]:
lamb.shape

torch.Size([5, 9])

In [4]:
cp_wf = [(np.array([-0.04190509,  0.00978352,  0.03772653]),
np.array([ 7.71566743e-06,  6.29104656e-06,  7.75028033e-01, -6.31926853e-01])),
 (np.array([-0.013862  , -0.0309698 ,  0.03706019]),
  np.array([ 2.23345856e-05, -2.27164750e-06,  9.94867341e-01,  1.01187815e-01])),
 (np.array([-0.07742409, -0.01736268,  0.03893098]),
  np.array([-2.27164750e-06, -2.23345856e-05,  1.01187815e-01, -9.94867341e-01]))]


In [8]:
np.array([np.concatenate(cpwf) for cpwf in cp_wf]).shape

(3, 7)

In [8]:
R_o_2_wf = Rotation.from_quat(obs['achieved_goal'][3:])
cp_list = []
for cp in cp_of:
    if cp is not None:
        c_pos, c_ori = cp
        c_pos_wf = R_o_2_wf.as_matrix() @ c_pos
        c_ori_wf = R_o_2_wf * Rotation.from_quat(c_ori)
        c_ori_wf = c_ori_wf.as_quat()
        cp_list.append((c_pos_wf, c_ori_wf))
    else:
        cp_list.append(None)

```python
        if use_actual_cp and self._current_contact_pos is not None:
            R_o_2_wf = Rotation.from_quat(obj_pose.orientation)
            R_w_2_of = R_o_2_wf.inv().as_matrix()
            flip_z = Rotation.from_rotvec([0, 0, np.pi]).as_matrix()
            for pos_wf, ori in zip(
                self._current_contact_pos, self._current_contact_ori
            ):
                if ori is None:
                    cp_list.append(None)
                else:
                    cp_quat = Rotation.from_matrix(R_w_2_of @ flip_z @ ori).as_quat()
                    R = Rotation.from_quat(obj_pose.orientation) * Rotation.from_quat(
                        cp_quat
                    )
                    quat_cp_2_w = R.as_quat()
                    cp_list.append((pos_wf, quat_cp_2_w))
        else:
            R_o_2_wf = Rotation.from_quat(obj_pose.orientation)
            for cp_param in cp_params:
                if cp_param is not None:
                    cp_of = c_utils.get_cp_of_from_cp_param(cp_param)
                    pos_wf = c_utils.get_wf_from_of(cp_of.pos_of, obj_pose)
                    R = R_o_2_wf * Rotation.from_quat(cp_of.quat_of)
                    quat_cp_2_w = R.as_quat()
                    # pos_wf = R_o_2_wf.as_matrix() @ cp_of.pos_of
                    cp_list.append((pos_wf, quat_cp_2_w))
```

In [9]:
cp_list

[(array([-0.04189374,  0.00980485, -0.59420026]),
  array([ 7.71566743e-06,  6.29104656e-06,  7.75028033e-01, -6.31926853e-01])),
 (array([-0.01385064, -0.03094847, -0.5948666 ]),
  array([ 2.23345856e-05, -2.27164750e-06,  9.94867341e-01,  1.01187815e-01])),
 (array([-0.07741273, -0.01734135, -0.5929958 ]),
  array([-2.27164750e-06, -2.23345856e-05,  1.01187815e-01, -9.94867341e-01]))]

In [10]:
prob = fop.BatchForceOptProblem()
G2 = prob.get_grasp_matrix(cp_list)

[-0.04189374  0.00980485 -0.59420026]
[-0.01385064 -0.03094847 -0.5948666 ]
[-0.07741273 -0.01734135 -0.5929958 ]


In [11]:
G == G2

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True]])

In [ ]:
op = obs['achieved_goal']
obj_pose = np.asarray([op for _ in range(10)])
cp_wf = [cp_wf for _ in range(10)]

In [41]:
env.pinocchio_utils.forward_kinematics(env.unwrapped.prev_observation['observation']['position'])

[[0.02326169384260848, 0.06405488240915017, 0.04545479707254271],
 [0.029158603370715852, -0.018415214062422455, 0.04492667906354321],
 [-0.04664714912920895, 0.0026628426942095426, 0.0449847631644697]]

In [42]:
[x[0] for x in cp_wf[0][0]]

[array([0.02032874, 0.05563401, 0.85349421]),
 array([ 0.02290265, -0.01268787,  0.85237941]),
 array([-0.03887879,  0.00640975,  0.85346884])]

In [19]:
env.close()

In [1]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x >= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])
bs = 128
A_tch = torch.randn(bs, m, n, requires_grad=True)
b_tch = torch.randn(bs, m, requires_grad=True)

# solve the problem
solution, = cvxpylayer(A_tch, b_tch)

# compute the gradient of the sum of the solution with respect to A, b
solution.sum().backward()